<a href="https://colab.research.google.com/github/aissahm/Unicamp-MO431/blob/master/4o_Exerc%C3%ADcio_de_casa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aissa Hadj Mohamed - RA 265189

# **4o Exercício de casa**

**Objective:**

Search for the best combinations of hyperparameters for the SVM for regression in a particular database.

First, we install the following library to measure the time of execution for each method selected in this exercise.


In [1]:
!pip install ipython-autotime

%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=8d0954a607831e15999928316cab170d10c0c32bd0d9b5becfc74112bfd735d3
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [95]:
# Imports
import numpy as np

X_train = np.load("/content/Xtreino5.npy")
Y_train = np.load("/content/ytreino5.npy")

X_test = np.load("/content/Xteste5.npy")
Y_test = np.load("/content/yteste5.npy")

time: 6.32 ms


In [0]:
print(X_train.shape)
print(Y_train.shape)

(404, 13)
(404,)


We have training and testing data of the same size.

# **SVM Regressor**

We will use the SVM from scikit-learn : https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html

We will use the "rbf" kernel with three parameters: C, gamma, and epsilon.
 
With:

1. C between 2^{-5} and 2^{15} (uniforme on the power element)
2. gamma between 2^{-15} and 2^{3} (uniforme on the power element)
3. epsilon entre 0.05 a 1.0 (uniforme on the power element)

**Measure for the error**

We will train the SVM on training data, with the combination of the parameters, and then measure the Mean Absolute Error (MAE).

# **Random search**

We first perform a search of the best parameters calibrations using Random Search from Sklearn, with 125 iterations.

Since we are supposed to validate the data on the test set, we use the library ShuffleSplit of Sklearn and set the number of split to 1. Since the model needs some validation data, we set the test size to 10% so that it can manage to train reasonably while not testing on a lot of the training data. The validation part should be performed on the test data X_test and Y_test.

In [191]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.stats import loguniform
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
from sklearn.model_selection import ShuffleSplit

svr = SVR(kernel='rbf')

C_range = loguniform(2**-5, 2**15) 
gamma_range = loguniform(2**-15, 2**3) 
epsilon_range = uniform(0.0, 1) 

cv_param = ShuffleSplit(n_splits=1, test_size = 0.1, train_size = 0.9)

distributions = {"C": C_range, "gamma": gamma_range, "epsilon":epsilon_range}
svr_search = RandomizedSearchCV(svr, param_distributions = distributions, cv=cv_param, scoring="neg_mean_absolute_error", n_iter=125)
search = svr_search.fit(X_train, Y_train)
search.best_params_

{'C': 7298.140587201118,
 'epsilon': 0.7337333214165747,
 'gamma': 5.040316708906656e-05}

time: 10.6 s


In [192]:
from sklearn.metrics import mean_absolute_error
svr = SVR(kernel='rbf', C=search.best_params_["C"], epsilon=search.best_params_["epsilon"], gamma=search.best_params_["gamma"])
svr.fit(X_train, Y_train)
Y_pred = svr.predict(X_test)
print("Lowest MAE value :", mean_absolute_error(Y_test, Y_pred))

Lowest MAE value : 2.5118252978845987
time: 932 ms


The best tuning for the hyperparameters is the following:

1. C : 7298.140587201118
2. gamma : 5.040316708906656e-05
3. epsilon : 0.7337333214165747

This combination minimizes the MAE to 2.5118252978845987.

The execution time was 10.6s.


# **Grid Search**

For the grid search, we generate randomly 5 values for each of C, gamma and epsilon, while respecting the distribution of each parameter as specified previously.

Since we are supposed to validate the data on the test set, we use the library ShuffleSplit of Sklearn and set the number of split to 1. Since the model needs some validation data, we set the test size to 10% so that it can manage to train reasonably while not testing on a lot of the training data. The validation part should be performed on the test data X_test and Y_test.

In [193]:
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform
from scipy.stats import loguniform
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR

C_range = loguniform.rvs(2**-5, 2**15, size = 5) 
gamma_range = loguniform.rvs(2**-15, 2**3, size = 5) 
epsilon_range = uniform.rvs(0.0, 1, size = 5)

cv_param = ShuffleSplit(n_splits=1, test_size = 0.1, train_size = 0.9)

param_grid = {"C": C_range, "gamma": gamma_range, "epsilon":epsilon_range}

svr = SVR(kernel='rbf')
grid = GridSearchCV(svr, param_grid=param_grid, cv=cv_param, scoring="neg_mean_absolute_error")
grid.fit(X_train, Y_train)

GridSearchCV(cv=ShuffleSplit(n_splits=1, random_state=None, test_size=0.1, train_size=0.9),
             error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([2.55698021e+00, 3.07100803e-01, 4.84514114e-02, 2.52605640e+02,
       2.21560036e+04]),
                         'epsilon': array([0.889845  , 0.21417028, 0.94975684, 0.24195105, 0.80097225]),
                         'gamma': array([3.34925235e-05, 4.09480770e+00, 3.91294686e-05, 4.55830441e-04,
       2.21201652e-03])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolute_error', verbose=0)

time: 31.2 s


In [194]:
from sklearn.metrics import mean_absolute_error

best = grid.best_params_
svr = SVR(kernel='rbf', C=best["C"], epsilon=best["epsilon"], gamma=best["gamma"])
svr.fit(X_train, Y_train)
Y_pred = svr.predict(X_test)

print("Best parameters")
print("C :" , best["C"])
print("gamma :" , best["gamma"])
print("epsilon :" , best["epsilon"])
print("Lowest MAE value :", mean_absolute_error(Y_test, Y_pred))

Best parameters
C : 22156.00357969096
gamma : 3.349252345653448e-05
epsilon : 0.9497568449541225
Lowest MAE value : 2.54618238214308
time: 1.71 s


The best tuning for the hyperparameters is the following: 

1. C : 22156.00357969096  , 
2. gamma : 3.349252345653448e-05 , 
3. epsilon : 0.9497568449541225. 

This combination minimizes the MAE to 2.54618238214308.

The execution time was 31.2s.

# **Bayesian Optimization**

We use the library from: https://github.com/hyperopt/hyperopt

In [160]:
!pip install hyperopt

time: 3.22 s


In [161]:
# define an objective function
from sklearn.metrics import mean_absolute_error

def objective(params):
    C_value = params["C"] 
    gamma_value= params["gamma"] 
    epsilon_value  = params["epsilon"]
    C_value = 2**C_value
    gamma_value = 2**gamma_value
    svr = SVR(kernel='rbf', C=C_value, epsilon = epsilon_value, gamma= gamma_value)
    svr.fit(X_train, Y_train)
    Y_pred = svr.predict(X_test)
    return mean_absolute_error(Y_test, Y_pred)

# define a search space
from hyperopt import hp
space = {
    'C': hp.uniform('C', -5, 15),
    'gamma': hp.uniform('gamma', -15, 3),
    'epsilon': hp.uniform('epsilon', 0.0, 1)
}

# minimize the objective over the space
from hyperopt import fmin, tpe, space_eval
best = fmin(objective, space, algo=tpe.suggest, max_evals=100)

100%|██████████| 100/100 [01:05<00:00,  1.54it/s, best loss: 2.3166163618840496]
time: 1min 5s


In [162]:
print(best)

{'C': 13.294998156076545, 'epsilon': 0.0011616828309439185, 'gamma': -14.751723567301333}
time: 1.95 ms


In [163]:
print("C :",2**best["C"], " , gamma :",  2** best["gamma"], ", epsilon :",  best["epsilon"])

C : 10050.62888784221  , gamma : 3.6248389723366706e-05 , epsilon : 0.0011616828309439185
time: 3.53 ms


In [164]:
svr = SVR(kernel='rbf', C=2**best["C"], epsilon = best["epsilon"], gamma=2** best["gamma"])
svr.fit(X_train, Y_train)
Y_pred = svr.predict(X_test)
print(mean_absolute_error(Y_test, Y_pred))

2.3166163618840496
time: 850 ms


The best tuning for the hyperparameters is the following: 
1. C : 10050.62888784221  , 
2. gamma : 3.6248389723366706e-05 , 
3. epsilon : 0.0011616828309439185. 

This combination minimizes the MAE to 2.3166163618840496.

The execution time was 1m5s.

# **PSO**

In [165]:
!pip install  pyswarm

  Created wheel for pyswarm: filename=pyswarm-0.6-cp36-none-any.whl size=4481 sha256=c528cf53601a2242316d9dd5700eefda349e59272780e879d1ba797407b5c5a2
  Stored in directory: /root/.cache/pip/wheels/37/c5/f6/b33b9ac00040cb95c1f00af982a4197334a672d6de43f4699f
Successfully built pyswarm
time: 4.8 s


In [166]:
from pyswarm import pso
from sklearn.metrics import mean_absolute_error

def objective(x):
    C_value, gamma_value, epsilon_value  = x
    C_value = 2**C_value
    gamma_value = 2**gamma_value
    svr = SVR(kernel='rbf', C= C_value, epsilon= epsilon_value, gamma= gamma_value)
    svr.fit(X_train, Y_train)
    Y_pred = svr.predict(X_test)
    return mean_absolute_error(Y_test, Y_pred)

C_lowerbound = -5
C_upperbound = 15

gamma_lowerbound = -15
gamma_upperbound = 3

epsilon_lowerbound = 0.0
epsilon_upperbound = 1.0

lb = [C_lowerbound, gamma_lowerbound, epsilon_lowerbound]
ub = [C_upperbound, gamma_upperbound, epsilon_upperbound]

xopt, fopt = pso(objective, lb, ub, swarmsize = 11, maxiter= 11)

Stopping search: maximum iterations reached --> 11
time: 3min 25s


In [167]:
print("Best parameters [C, gamma, epsilon]: ", xopt, "Lowest MAE reached value: " ,fopt )

Best parameters [C, gamma, epsilon]:  [ 14.05436066 -15.           0.02925631] Lowest MAE reached value:  2.2742858413991613
time: 1.71 ms


In [168]:
print("C :",2**xopt[0], " , gamma :",  2** xopt[1], ", epsilon :",  xopt[2])

C : 17013.126423903264  , gamma : 3.0517578125e-05 , epsilon : 0.029256306782726723
time: 2.2 ms


The best tuning for the hyperparameters is the following:

1. C : 17013.126423903264  , 
2. gamma : 3.0517578125e-05 , 
3. epsilon : 0.029256306782726723. 

This combination minimizes the MAE to 2.2742858413991613.

The execution time was 3min and 25s.

# **Simulated annealing**

In [0]:
from scipy.optimize import dual_annealing
from sklearn.svm import SVR
import numpy as np

from sklearn.metrics import mean_absolute_error

def objective(params):
    C_value = params[0] 
    gamma_value= params[1] 
    epsilon_value  = params[2]
    C_value = 2**C_value
    gamma_value = 2**gamma_value
    svr = SVR(kernel='rbf', C=C_value, epsilon = epsilon_value, gamma= gamma_value)
    svr.fit(X_train, Y_train)
    Y_pred = svr.predict(X_test)
    return mean_absolute_error(Y_test, Y_pred)

C_lowerbound = -5
C_upperbound = 15

gamma_lowerbound = -15
gamma_upperbound = 3

epsilon_lowerbound = 0.1
epsilon_upperbound = 1.0

lw = [C_lowerbound, gamma_lowerbound, epsilon_lowerbound]
up = [C_upperbound, gamma_upperbound, epsilon_upperbound]

ret = dual_annealing(objective, bounds=list(zip(lw, up)), no_local_search=True, maxiter = 125)

print("global minimum: xmin = {0}, f(xmin) = {1:.6f}".format(ret.x, ret.fun))

global minimum: xmin = [ 13.21934282 -14.86303894   0.32641787], f(xmin) = 2.313020
time: 3min 42s


In [0]:
print("C :",2**ret.x[0], " , gamma :",  2** ret.x[1], ", epsilon :",  ret.x[2])

C : 9537.152144403466  , gamma : 3.3556716086310645e-05 , epsilon : 0.3264178696524723
time: 1.25 ms


In [0]:
svr = SVR(kernel='rbf', C=2**ret.x[0], epsilon = ret.x[2], gamma= 2**ret.x[1])
svr.fit(X_train, Y_train)
Y_pred = svr.predict(X_test)
print(mean_absolute_error(Y_test, Y_pred))

2.3130198794458274
time: 746 ms


The best tuning for the hyperparameters is the following:

1. C : 9537.152144403466  , 
2. gamma : 3.3556716086310645e-05 , 
3. epsilon : 0.3264178696524723. 

This combination minimizes the MAE to 2.3130198794458274.

The execution time was 3min and 42s.

# **CMA**

https://github.com/CyberAgent/cmaes

In [0]:
pip install cma

     |████████████████████████████████| 235kB 8.5MB/s 
time: 4.33 s


In [0]:
import cma
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
import numpy as np

def objective(x):
    C_value = 2**( -5 + x[0]*20 )
    gamma_value = 2**( -15 + x[1]*18 )
    svr = SVR(kernel='rbf', C= C_value, epsilon=abs( x[2] ), gamma= gamma_value)
    svr.fit(X_train, Y_train)
    Y_pred = svr.predict(X_test)
    return mean_absolute_error(Y_test, Y_pred)

C_lowerbound = 0
C_upperbound = 1

gamma_lowerbound = 0
gamma_upperbound = 1

epsilon_lowerbound = 0
epsilon_upperbound = 1

lw_bound = [C_lowerbound, gamma_lowerbound, epsilon_lowerbound]
up_bound = [C_upperbound, gamma_upperbound, epsilon_upperbound]

x0 = 3 * [0.1]
sigma = 0.25

es = cma.CMAEvolutionStrategy(x0, sigma, {'bounds': [lw_bound, up_bound]})
es.optimize(objective, iterations=125)

(3_w,7)-aCMA-ES (mu_w=2.3,w_1=58%) in dimension 3 (seed=1067096, Fri May  8 14:55:24 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      7 4.424422987071383e+00 1.0e+00 2.53e-01  2e-01  3e-01 0:00.1
    2     14 3.742507117802927e+00 1.3e+00 3.27e-01  2e-01  5e-01 0:00.3
    3     21 3.090390348611081e+00 2.0e+00 3.32e-01  2e-01  4e-01 0:00.6
    4     28 3.662712471406125e+00 1.9e+00 3.04e-01  2e-01  3e-01 0:06.2
    6     42 3.666972451250105e+00 1.8e+00 2.52e-01  1e-01  3e-01 0:10.4
   14     98 2.433121397728236e+00 4.0e+00 4.81e-01  2e-01  7e-01 0:22.3
   15    105 2.422312361566948e+00 4.5e+00 4.64e-01  2e-01  6e-01 0:31.8
   16    112 2.307529728740064e+00 5.2e+00 4.27e-01  2e-01  6e-01 0:42.4
   17    119 2.359273515119458e+00 5.9e+00 4.53e-01  1e-01  7e-01 0:50.4
   18    126 2.298630905749822e+00 7.1e+00 4.29e-01  1e-01  7e-01 1:00.9
   20    140 2.359812016852811e+00 8.1e+00 2.96e-01  7e-02  5e-01 1:19.2
   21    147 2.287330363314429e+00

time: 20min 23s


In [0]:
es.result_pretty()

final/bestever f-value = 2.274515e+00 2.273549e+00
incumbent solution: [0.9525847773005284, 3.173825733309842e-05, 0.03338951973563586]
std deviation: [0.00014077014242429176, 0.0010986362431870242, 0.0012248489143510115]


CMAEvolutionStrategyResult(xbest=array([9.52402289e-01, 1.88414871e-05, 3.16117478e-02]), fbest=2.2735490333614745, evals_best=636, evaluations=875, iterations=125, xfavorite=array([9.52584777e-01, 3.17382573e-05, 3.33895197e-02]), stds=array([0.00014077, 0.00109864, 0.00122485]), stop={})

time: 8.88 ms


In [0]:
C_value = 2**( -5 + 0.9525847773005284*20 )
gamma_value = 2**( -15 +  3.173825733309842e-05*18 )

print(C_value, gamma_value)

16981.726782954633 3.052966508538199e-05
time: 1.38 ms


In [0]:
svr = SVR(kernel='rbf', C= C_value, epsilon=abs( 0.03338951973563586 ), gamma= gamma_value)
svr.fit(X_train, Y_train)
Y_pred = svr.predict(X_test)
print(mean_absolute_error(Y_test, Y_pred))

2.275273646800402
time: 1.34 s


The best tuning for the hyperparameters is the following:

1. C : 16981.726782954633 
2. gamma : 3.052966508538199e-05 
3. epsilon : 0.03338951973563586 

This combination minimizes the MAE to 2.275273646800402.

The execution time was 20min and 23s.

# **Conclusion**

PSO and CMA gave the best resuults in terms of Mean Absolute Error. Both methods managed to reduce it to around 2.27 . However, it took them some considerable amount of time (1min and 5 sec, and 20 min and 23 sec respectively). 

The fastest in execution time was the Random Search method, it took 10s to execute. However, it managed to reduce the MAE value to 2.51 (second worst result among all the methods). 

The Bayezian optimization reached more or less the same value for MAE, around 2.31 , as the Simulated Annealing method. However, it took it much less time (1min 5sec vs 3min 42 sec).

The Grid Search took 31sec to minimizes the value of MAE to about 2.54. However, we needed to simulate the Grid Search many times before reaching a low value of 2.54. 125 combinations for the hyperparameters C, gamma and epsilon, was too low. We need to generate randomly more numbers for each parameter (instead of only 5 for each). But this would naturally increase the time to execute this search method for tuning.

Depending on which MAE value is acceptable, someone could choose PSO method to reach a very low value of MAE but would have to spend some time for this method to execute. Or, they could select the Random Search method for much faster tuning of the hyperparaemeters, but they would risk getting a higher MAE value.
